In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

%matplotlib inline

PATH = '../'

In [2]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.61 s


### LB Score Calc

Для подсчета скора просто используйте функцию getLBScore

In [3]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

In [4]:
def getLBScore(real, predicted):
    smp = []
    for sat_id in real['sat_id'].unique():
        rv = real[real['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        sym = predicted[predicted['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        smp.append(smape(sym.ravel(), rv.ravel()))
    meanSmape = np.mean(smp)
    lbScore = 100 * (1 - meanSmape)
    print('Mean SMAPE ', meanSmape, '; LB score ', lbScore)

In [5]:
%%time
# LB ~ 66
# Скор при использовании симулирующих значений на трейне как предсказаний
trainSym = train.copy()
trainSym.iloc[:, 3:9] = trainSym.iloc[:, 9:].values
getLBScore(train, trainSym)

Mean SMAPE  0.15090231392579082 ; LB score  84.90976860742092
Wall time: 2.67 s


In [6]:
%%time
# Скор при использовании симулирующих значений на трейне как предсказаний только для айдишников из теста
trainTestSatellites = train['sat_id'].unique()[[True if sat_id in test['sat_id'].unique() else False for sat_id in train['sat_id'].unique()]]
trainTestData = train.loc[[True if sat_id in trainTestSatellites else False for sat_id in train['sat_id'].values], :]

trainSym = trainTestData.copy()
trainSym.iloc[:, 3:9] = trainSym.iloc[:, 9:].values

getLBScore(trainTestData, trainSym)

Mean SMAPE  0.14887327646453175 ; LB score  85.11267235354683
Wall time: 4.33 s


In [7]:
%%time
# Скор при полностью правильном предсказании
getLBScore(train, train)

Mean SMAPE  0.0 ; LB score  100.0
Wall time: 2.2 s


In [8]:
%%time
# Скор при рандомном предсказании
trainRnd = train.copy()
trainRnd.iloc[:, 3:9] = np.random.rand(*train.iloc[:, 3:9].shape)
getLBScore(train, trainRnd)

Mean SMAPE  0.88087618138585 ; LB score  11.912381861415
Wall time: 2.84 s


In [9]:
# LB ~ 32
# Половинку значений зануляем
trainSym = train.copy()
trainSym.iloc[:, 3:9] = trainSym.iloc[:, 9:].values
trainSym.iloc[157810:, 3:9] = 0
getLBScore(train, trainSym)

Mean SMAPE  0.7728933735292876 ; LB score  22.710662647071246


### Model training

In [7]:
training_columns = ['epoch', 'sat_id', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
prediction_columns = ['x','y','z','Vx', 'Vy', 'Vz']
X = train.loc[:, training_columns]
y = train.loc[:, prediction_columns]
Xtest = test.loc[:, training_columns]

In [8]:
def extractTimeFeats(data):
    timeFeats = data['epoch'].apply(lambda x: str(x).split('T')[1].split('.')[0].split(':')).values
    h,m,s = [],[],[]
    for feat in timeFeats:
        h.append(feat[0]); m.append(feat[1]); s.append(feat[2])
    return h,m,s

In [9]:
h,m,s = extractTimeFeats(X)
X.loc[:, 'h'] = list(map(int, h)); X.loc[:, 'm'] = list(map(int, m)); X.loc[:, 's'] =list(map(int, s))
X.drop(columns = 'epoch', inplace = True)

h,m,s = extractTimeFeats(Xtest)
Xtest.loc[:, 'h'] = list(map(int, h)); Xtest.loc[:, 'm'] = list(map(int, m)); Xtest.loc[:, 's'] = list(map(int, s))
Xtest.drop(columns = 'epoch', inplace = True)

In [10]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

models = {
      'logreg': LinearRegression(), 
      'lasso': Lasso(),
      'ridge': Ridge(),
#       'mlp': MLPRegressor(), 
      #'knn': KNeighborsRegressor(), 
      #'dtree': DecisionTreeRegressor(), 
      #'rfc': RandomForestRegressor(), 
      #'gbm': GradientBoostingRegressor()
     }

trainSize = 0.8
for name, model in models.items():
    print(f'Running model {name}...')
    submissionValid = train.iloc[int(X.shape[0] * 0.8):, :].copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
    Xtr, Xval = X.iloc[:int(X.shape[0] * 0.8), :], X.iloc[int(X.shape[0] * 0.8):, :]
    Ytr, Yval = y.iloc[:int(X.shape[0] * 0.8), :], y.iloc[int(X.shape[0] * 0.8):, :]
    for predValue in ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']:
        ytr,yval = Ytr[predValue].values, Yval[predValue].values
        model.fit(Xtr, ytr)
        ypred = model.predict(Xval)
        print(f'SMAPE on {predValue} ', smape(ypred, yval))
        submissionValid[[predValue]] = ypred
    getLBScore(train.iloc[int(X.shape[0] * 0.8):, :],submissionValid)
    print('\n-----------------------------------\n')

Running model logreg...
SMAPE on x  0.22296569333138377
SMAPE on y  0.22845106300166876
SMAPE on z  0.2446033448681394
SMAPE on Vx  0.25199900845893514
SMAPE on Vy  0.2641166470849824
SMAPE on Vz  0.2743765095739411
Mean SMAPE  0.21712844793677782 ; LB score  78.28715520632221

-----------------------------------

Running model lasso...
SMAPE on x  0.2229544351216954
SMAPE on y  0.22844113898715993
SMAPE on z  0.2445933787161466
SMAPE on Vx  0.303768583047601
SMAPE on Vy  0.3188240309702224
SMAPE on Vz  0.3676622512712268
Mean SMAPE  0.25251954718511316 ; LB score  74.74804528148869

-----------------------------------

Running model ridge...
SMAPE on x  0.22296569124354532
SMAPE on y  0.22845106123018344
SMAPE on z  0.24460333566057743
SMAPE on Vx  0.25199907670918625
SMAPE on Vy  0.26411671962487066
SMAPE on Vz  0.2743766289223741
Mean SMAPE  0.2171284930796368 ; LB score  78.28715069203632

-----------------------------------



Реальный скор того что внизу - 64.63

In [11]:
# LB ~ 64
models = {
    'ridge': Ridge()
}

trainSize = 0.9
for name, model in models.items():
    print(f'Running model {name}...')
    Xtr, Xval = train_test_split(X, train_size = trainSize)
    Ytr, Yval = y.iloc[Xtr.index], y.iloc[Xval.index]
    submissionValid = train.iloc[Xval.index, :].copy()[['id', 'sat_id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']]
    for predValue in ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']:
        ytr,yval = Ytr[predValue].values, Yval[predValue].values
        model.fit(Xtr, ytr)
        ypred = model.predict(Xval)
        print(f'SMAPE on {predValue} ', smape(ypred, yval))
        submissionValid[[predValue]] = ypred
        submission[[predValue]] = model.predict(Xtest)
    getLBScore(train.iloc[Xval.index, :],submissionValid)
    print('\n-----------------------------------\n')

Running model ridge...


C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


SMAPE on x  0.19844223749190634
SMAPE on y  0.20071991411184406
SMAPE on z  0.22925405083889078
SMAPE on Vx  0.2347656621558797
SMAPE on Vy  0.23685419354462256
SMAPE on Vz  0.2741054789896423
Mean SMAPE  0.20126387663261341 ; LB score  79.87361233673866

-----------------------------------



In [67]:
# Реальный скор: 64.63, скор при трейн тест сплите 56-59.66
submission.to_csv('baseline.csv', index = None)